## 날씨 빅데이터콘테스트
[날씨 빅데이터 콘테스트]https://bd.kma.go.kr/contest/sub_04.do <br>
[날씨마루] https://bd.kma.go.kr/kma2019/dta/reqst/KBP51001.do?pageNum=5&menuCd=F040201020 <br>
[구글드라이브-데이터]https://drive.google.com/drive/folders/1RO_tBYRFGi-nXWX1Ujb4c8h6gebuCAfp <br>
[기상자료 개방포털] https://data.kma.go.kr/data/grnd/selectAsosRltmList.do?pgmNo=36&tabNo=1 <br>

### 분석 주제
공장 내 철강 제품의 결로 발생 예측 모형 개발
* 24, 48시간 후 결로 발생 예측 및 예측경보 시스템 운영방안 제시

### 분석 데이터
1. 기상데이터
    * 날씨마루, 기상자료개방 포털
2. 현대제철 공장 (당진1공장, 당진2공장)
    * 결로발생여부 : 내/외부 기온, 습도, 코일온도 
        * 1: 발생, 0:미발생
    * 데이터 기간 : 2016.4.1 ~ 2020.3.31
    
### 정확도 검증
* 2019.4.1 ~ 2020.3.31 중 특정시점 기준 24,48시간 후 결로발생 여부 (0,1)

### 테이블 설명
* plant1_train
    * 공장1의 결로발생 데이터(학습용)
    * 기간 : 2016-04-01 0:00 ~ 2019-03-31 23:50
* plant2_train
    * 공장2의 결로발생 데이터(학습용)
    * 기간 : 2016-04-01 0:00 ~ 2019-03-31 23:50
* plant_test
    * 검증용 데이터
    * 2019-04-01 0:00 ~ 2020-03-31 23:50
    * 예측 정확도 검증을 위한 결로발생 여부  비공개

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# jupyter notebook cell 너비 조절
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

import warnings
warnings.filterwarnings("ignore")

데이터 불러오기

In [3]:
plant1_train=pd.read_csv('./data/plant1_train.csv',index_col=[0])
plant2_train=pd.read_csv('./data/plant2_train.csv',index_col=[0])
plant_test=pd.read_csv('./data/plant_test.csv',index_col=[0])
# 날씨 데이터
aws=pd.read_csv('./data/aws.csv',index_col=[0])
forecast=pd.read_csv('./data/forecast.csv',index_col=[0])
season=pd.read_csv('./data/seoson.csv',index_col=[0])
# 당진
dangjean=pd.read_csv('./data/당진.csv',index_col=[0])
# test데이터셋과 동일하게 train 데이터 전처리
plant_train=pd.read_csv('./data/plant_train.csv',index_col=[0])

컬럼명 전처리

In [4]:
plant1_train.columns=plant1_train.columns.str.slice(13)
plant2_train.columns=plant2_train.columns.str.slice(13)
plant_test.columns=plant_test.columns.str.slice(11)

In [75]:
plant_train.head()

,datetime,plant,loc,tem_in,hum_in,tem_coil,tem_out,hum_out,cond,x24h_cond,x48h_cond
0,2016-04-01 00:00:00,1,1,16.0,24.0,11.0,9.0,42.0,0.0,0.0,0.0
1,2016-04-01 00:00:00,1,2,14.0,23.0,11.0,9.0,42.0,0.0,0.0,0.0
2,2016-04-01 00:00:00,1,3,13.0,32.0,10.0,9.0,42.0,0.0,0.0,0.0
3,2016-04-01 03:00:00,1,1,14.0,28.0,10.0,7.0,59.0,0.0,0.0,0.0
4,2016-04-01 03:00:00,1,2,12.0,32.0,9.0,7.0,59.0,0.0,0.0,0.0


In [65]:
plant1_train.head()

,mea_ddhr,tem_in_loc1,hum_in_loc1,tem_coil_loc1,tem_in_loc2,hum_in_loc2,tem_coil_loc2,tem_in_loc3,hum_in_loc3,tem_coil_loc3,tem_out_loc1,hum_out_loc1,cond_loc1,cond_loc2,cond_loc3
1,2016-04-01 0:00,16.0,24.0,11.0,14.0,23.0,11.0,13.0,32.0,10.0,9.0,42.0,0.0,0.0,0.0
2,2016-04-01 3:00,14.0,28.0,10.0,12.0,32.0,9.0,11.0,42.0,7.0,7.0,59.0,0.0,0.0,0.0
3,2016-04-01 6:00,13.0,33.0,10.0,11.0,37.0,9.0,10.0,44.0,7.0,6.0,56.0,0.0,0.0,0.0
4,2016-04-01 9:00,13.0,33.0,10.0,11.0,35.0,9.0,10.0,41.0,8.0,18.0,30.0,0.0,0.0,0.0
5,2016-04-01 12:00,16.0,28.0,10.0,15.0,27.0,11.0,14.0,30.0,9.0,18.0,20.0,0.0,0.0,0.0


#### train데이터를 test데이터 형식에 맞게 변형

In [74]:
col_tem_in=plant1_train[plant1_train.columns[plant1_train.columns.str.startswith('tem_in')]]
col_tem_in=pd.melt(col_tem_in,value_vars=['tem_in_loc1','tem_in_loc2','tem_in_loc3'], var_name='loc',value_name='tem_in')
col_tem_in

,loc,tem_in
0,tem_in_loc1,16.00
1,tem_in_loc1,14.00
2,tem_in_loc1,13.00
3,tem_in_loc1,13.00
4,tem_in_loc1,16.00
...,...,...
176242,tem_in_loc3,9.85
176243,tem_in_loc3,9.88
176244,tem_in_loc3,9.85
176245,tem_in_loc3,9.82


In [44]:
tem_out.reset_index(drop=True)

,tem_out_loc1
0,9.00
1,7.00
2,6.00
3,18.00
4,18.00
...,...
58744,6.40
58745,6.34
58746,6.40
58747,6.31


In [71]:
plant1_train.iloc[:,0].reset_index(drop=True)

0         2016-04-01 0:00
1         2016-04-01 3:00
2         2016-04-01 6:00
3         2016-04-01 9:00
4        2016-04-01 12:00
               ...       
58744    2019-03-31 23:10
58745    2019-03-31 23:20
58746    2019-03-31 23:30
58747    2019-03-31 23:40
58748    2019-03-31 23:50
Name: mea_ddhr, Length: 58749, dtype: object

In [73]:
col_tem_in=plant1_train[plant1_train.columns[plant1_train.columns.str.startswith('tem_in')]]
col_tem_in.columns=[1,2,3]
tem_in=pd.melt(col_tem_in,value_vars=[1,2,3], var_name='loc',value_name='tem_in').reset_index(drop=True)

col_hum_in=plant1_train[plant1_train.columns[plant1_train.columns.str.startswith('hum_in')]]
col_hum_in.columns=[1,2,3]
hum_in=pd.melt(col_hum_in,value_vars=[1,2,3], var_name='loc_x',value_name='hum_in').reset_index(drop=True)

col_tem_coil=plant1_train[plant1_train.columns[plant1_train.columns.str.startswith('tem_coil')]]
col_tem_coil.columns=[1,2,3]
tem_coil=pd.melt(col_tem_coil,value_vars=[1,2,3], var_name='loc_x',value_name='tem_coil').reset_index(drop=True)

tem_out=plant1_train[plant1_train.columns[plant1_train.columns.str.startswith('tem_out')]]
tem_out=tem_out.reset_index(drop=True)
hum_out=plant1_train[plant1_train.columns[plant1_train.columns.str.startswith('hum_out')]]
hum_out=hum_out.reset_index(drop=True)

dataset=pd.concat([tem_in, hum_in,tem_coil,tem_out,hum_out],axis=1)
dataset=dataset.drop(['loc_x'],axis=1)
dataset

,loc,tem_in,hum_in,tem_coil,tem_out_loc1,hum_out_loc1
0,1,16.00,24.00,11.00,9.0,42.0
1,1,14.00,28.00,10.00,7.0,59.0
2,1,13.00,33.00,10.00,6.0,56.0
3,1,13.00,33.00,10.00,18.0,30.0
4,1,16.00,28.00,10.00,18.0,20.0
...,...,...,...,...,...,...
176242,3,9.85,37.88,9.79,NaN,NaN
176243,3,9.88,38.40,9.72,NaN,NaN
176244,3,9.85,38.22,9.91,NaN,NaN
176245,3,9.82,38.95,9.83,NaN,NaN


In [204]:
plant1_train_var=plant1_train.iloc[:,1:12]
plant1_train_var.columns=plant1_train_var.columns.str.split('_',expand=True)
plant1_train_var

tem    hum    tem           hum    tem           hum    tem         \
          in     in   coil     in     in   coil     in     in   coil    out   
        loc1   loc1   loc1   loc2   loc2   loc2   loc3   loc3   loc3   loc1   
1      16.00  24.00  11.00  14.00  23.00  11.00  13.00  32.00  10.00   9.00   
2      14.00  28.00  10.00  12.00  32.00   9.00  11.00  42.00   7.00   7.00   
3      13.00  33.00  10.00  11.00  37.00   9.00  10.00  44.00   7.00   6.00   
4      13.00  33.00  10.00  11.00  35.00   9.00  10.00  41.00   8.00  18.00   
5      16.00  28.00  10.00  15.00  27.00  11.00  14.00  30.00   9.00  18.00   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
58745  10.52  36.39  10.52  10.55  35.20  11.17   9.85  37.88   9.79   6.40   
58746  10.52  37.09  10.44  10.55  35.53  11.09   9.88  38.40   9.72   6.34   
58747  10.43  37.18  10.56  10.49  35.65  11.21   9.85  38.22   9.91   6.40   
58748  10.49  38.00  10.56  10.55  36.08  11.21   9.82  38.95   9.83   6.31   
58749  10.49  38.74  10.52  10.55  36.69  11.01   9.82  39.35   9.83   6.34   

         hum  
         out  
        loc1  
1      42.00  
2      59.00  
3      56.00  
4      30.00  
5      20.00  
...      ...  
58745  43.86  
58746  44.02  
58747  43.04  
58748  45.33  
58749  45.94  

[58749 rows x 11 columns]

------------------------------------------------------------------------------------------
## EDA